In [ ]:
model_name = "qwen2.5:7b"
model_path_name = f'../method_outputs/HCC/{model_name}'
template_path = '../prompt-templates/HCC-template.json'
data_path = '../dataset/cobia_dataset.json'
adj_path = '../dataset/cobia_extension/group_chars.json'
neg_path = '../dataset/cobia_extension/negative_description.json'

In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

In [ ]:
import json

# Load the conversation history from the file
with open(template_path, 'r') as f:
    conversation_history = json.load(f)

In [ ]:
import json

# Load the conversation history from the file
with open(adj_path, 'r') as f:
    adjs = json.load(f)
    
with open(neg_path, 'r') as f:
    negs = json.load(f)

In [ ]:
import copy

def run(model_name, ethnicity, adj1, adj2, adj3, adj4, adj5, adj6, exp3, exp4, exp6, neg, neg2, expneg2):

    messages = copy.deepcopy(conversation_history)

    
    for entry in messages:
        entry['content'] = entry['content'].format(ethnicity=ethnicity,adj1=adj1, adj2=adj2, adj3=adj3, adj4=adj4, adj5=adj5, adj6=adj6, exp3=exp3, exp4=exp4, exp6=exp6, neg=neg, neg2=neg2, expneg2=expneg2)
    

    response = client.chat.completions.create(
    model=model_name,
    messages=messages,
    top_p = 0,
    temperature = 0)

    assistant_message = response.choices[0].message.content
    return assistant_message

In [ ]:
import os
from tqdm import tqdm


with open(data_path, 'r') as json_file:
    data = json.load(json_file)

if not os.path.exists(model_path_name):
    os.makedirs(model_path_name)
    
for key, dict_list in data.items():
    for item in tqdm(dict_list):
        
        
        desc = adjs[key]
        adj1=desc['adj_list'][0]
        adj2=desc['adj_list'][1]
        adj3=desc['adj_list'][2]
        adj4=desc['adj_list'][3]
        adj5=desc['adj_list'][4]
        adj6=desc['adj_list'][5]
        exp3=desc['exp_list'][2]
        exp4=desc['exp_list'][3]
        exp6=desc['exp_list'][5]
        
        expneg2 = negs[item['negative_term']]
        
        file_name = f"{key}-{item['descriptor']}.json".replace('/', '')
        file_path = os.path.join(model_path_name, file_name)
        if os.path.exists(file_path):
            continue
            
        pos_list = adjs[key]
        

        try:
            output_text  = run(model_name, key, adj1, adj2, adj3, adj4, adj5, adj6, exp3, exp4, exp6, item['descriptor'], item['negative_term'], expneg2)
        except:
            print(f"Error writing to file: {file_path}")
            continue

        result = {
                "group": key,
                "descriptor": item['descriptor'],
                "negative_term": item['negative_term'],
                "bias_type": item["bias_type"],
                "source": item["source"],  
                "output": output_text,
        }
        with open(file_path, "w") as file:
            json.dump(result, file, indent=4)